In [ ]:
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime, timezone, timedelta
from custom_process_domain import process_domain_normal
import matplotlib.pyplot as plt
from zoneinfo import ZoneInfo
import seaborn
seaborn.set()



## User Input

In [ ]:
POSSIBLE_USERS = { # Just to make sure we're good
  "Juan": "Juan",
  "Nathan": "Nathan",
  "Sanyam": "Sanyam",
  "Nipun": "Nipun",
  "Parsa": "Parsa",
}

# User Input
USER = POSSIBLE_USERS["Sanyam"] # Change this line to one of the names above

# START_DATE
START_DATE_YEAR = 2022
START_DATE_MONTH = 9
START_DATE_DAY = 19

WEEKS_TO_INCREMENT = 0 # Add WEEKS_TO_INCREMENT * 7 days to START_TIME
DATE_INTERVAL_TYPE = 'month' # 'weekend', 'week', 'all_week', 'all_time

##############################
# Graphs
##############################

# Save Graphs
SAVE_GRAPH_1 = False
SAVE_GRAPH_2 = False
SAVE_GRAPH_3 = False
SAVE_GRAPH_4 = False
SAVE_GRAPH_5 = True
SAVE_GRAPH_6 = True

##############################
# Options
##############################
MIN_CHROME_VISIT_DURATION = 5
MAX_CHROME_VISIT_DURATION = 2.88e+10 # 8 hours in microseconds

MIN_SAFARI_SCORE = 5

### Config per User

In [ ]:
if USER == "Nathan":
  DB_FILE = "../data/history--2022-10-21--Nathan-Tsai.sqlite3"
  RATING_FILE = "../rated-output/Ratings - CMPT 353 - Nathan Tsai (1).csv"
  USER_TIMEZONE_STRING = 'US/Pacific'
elif USER == "Juan":
  DB_FILE = "../data/history--2022-10-21--Juan-Gonzalez.sqlite3"
  RATING_FILE = "../rated-output/Ratings - CMPT 353 - Juan Gonzalez.csv"
  USER_TIMEZONE_STRING = 'US/Pacific'
elif USER == "Sanyam":
  DB_FILE = "../data/history--2022-11-25--Sanyam-Safari.db"
  RATING_FILE = "../rated-output/Ratings of Inputted Users - CMPT 353 - Sanyam (2).csv"
  USER_TIMEZONE_STRING = 'US/Pacific'
elif USER == "Nipun":
  DB_FILE = "../data/history--2022-11-26--Nipun-Safari.db"
  RATING_FILE = "../rated-output/Ratings of Inputted Users - CMPT 353 - Nipun.csv"
  USER_TIMEZONE_STRING = 'Asia/Colombo' # India standard time: UTC+5:30
elif USER == "Parsa":
  DB_FILE = "../data/history--2022-12-05--Pasha.sqlite3"
  RATING_FILE = "../rated-output/Ratings of Inputted Users - CMPT 353 - Pasha.csv"
  USER_TIMEZONE_STRING = 'Asia/Colombo' # India standard time: UTC+5:30
else:
  assert(False and "User name is invalid")

USER_TIMEZONE = ZoneInfo(USER_TIMEZONE_STRING)
TIMEZONE_STRING = USER_TIMEZONE_STRING


START_TIME = datetime(
  START_DATE_YEAR,
  START_DATE_MONTH,
  START_DATE_DAY, 
  tzinfo=USER_TIMEZONE
) # the data to look at. Should be a Monday

# The values per user productivity
productivity_to_int_map = {
  'Always Distracted' : -2,
  'Mostly Distracted' : -1,
  'Neutrel' : 0,
  'Mostly Intentional' : 1,
  'Always Intentional' : 2,
}

## Archive

### Format User Input for Graphs / Files

In [ ]:
# Increment by a number of weeks
START_TIME = START_TIME + timedelta(days=7 * WEEKS_TO_INCREMENT)

# Add 1 day if weekend, otherwise use weekday
date_types = {
  'weekend': 1,
  'week': 4,
  'all_week': 6,
  'month': 30,
  'all_time': 7 * 52 * 3, # 3 years
}
DAYS_TO_ADD = date_types[DATE_INTERVAL_TYPE]

if DATE_INTERVAL_TYPE == 'weekend':
  START_TIME = START_TIME + timedelta(days=-2)

END_TIME = START_TIME + timedelta(days=DAYS_TO_ADD)


In [ ]:
if DATE_INTERVAL_TYPE == 'all_time':
  start_date_for_graph = START_TIME.strftime('%a, %b %d %Y')
else:
  start_date_for_graph = START_TIME.strftime('%a, %b %d')

  
end_date_for_graph = END_TIME.strftime('%a, %b %d, %Y')
time_for_graph = "{} - {}".format(start_date_for_graph, end_date_for_graph)
CUSTOM_PARAMS_FOR_GRAPH = "{}: {}".format(USER, time_for_graph)

start_date_for_file = START_TIME.strftime('%b-%d-%Y')
end_date_for_file = END_TIME.strftime('%b-%d-%Y')
time_for_file = "{}--{}".format(start_date_for_file, end_date_for_file)
CUSTOM_PARAMS_FOR_FILE = "{}--{}".format(USER, time_for_file)

In [ ]:
# Constants
WINDOWS_EPOCH_MICROSECS      = -11644473600000 * 1000
SAFARI_TIME_UPDATE           = 978307200

In [ ]:
# Process the type of Database
TYPE_IS_SAFARI = "TYPE_IS_SAFARI"
TYPE_IS_CHROME = "TYPE_IS_CHROME"
TYPE_OF_DB = {
  "db": TYPE_IS_SAFARI,
  "sqlite": TYPE_IS_CHROME,
  "sqlite3": TYPE_IS_CHROME,
}

extension = DB_FILE.split(".")[-1]
DB_TYPE = TYPE_OF_DB[extension]

### Helpful Functions

In [ ]:
NUMBER_OF_INCREMENTS = 24 * 2
DAY_IN_SECONDS = 60 * 60 * 24
ROUND_TO = DAY_IN_SECONDS / NUMBER_OF_INCREMENTS

def get_half_hour(data: pd.Series) -> pd.Series:
  return (data / ROUND_TO).round().astype(np.int32)

### Bunch of Processing

#### Load History Database

In [ ]:
if DB_TYPE == TYPE_IS_CHROME:
  query_get_urls_and_times = """
  SELECT v.id, v.visit_time, v.visit_duration, u.url
  FROM 'visits' as v 
  LEFT JOIN urls u ON u.id = v.url
  """

  TIME_CORRECTION_TO_ADD_TO_VISIT_TIME = WINDOWS_EPOCH_MICROSECS
elif DB_TYPE == TYPE_IS_SAFARI:
  query_get_urls_and_times = """
  SELECT v.id, v.visit_time, v.score, u.url
  FROM 'history_visits' as v 
  LEFT JOIN history_items u ON u.id = v.history_item
  """
  TIME_CORRECTION_TO_ADD_TO_VISIT_TIME = SAFARI_TIME_UPDATE
else:
  assert(False and "Extension of database is invalid")

In [ ]:
with sqlite3.connect(DB_FILE) as con:
  visits = pd.read_sql_query(query_get_urls_and_times, con)

visits.head()

#### Process the time

In [ ]:
# Visit time in microseconds (s/1,000,000)
# https://chromium.googlesource.com/chromium/src/+/lkgr/base/time/time.h

if DB_TYPE == TYPE_IS_CHROME:
  visit_time_in_ns = (visits['visit_time'] + TIME_CORRECTION_TO_ADD_TO_VISIT_TIME) * 1000
  visits['visit_time_epoch'] = pd.to_datetime(visit_time_in_ns, unit='ns', utc=True).map(lambda x: x.tz_convert(TIMEZONE_STRING))
elif DB_TYPE == TYPE_IS_SAFARI:
  visit_time_in_ns = (visits['visit_time'] + TIME_CORRECTION_TO_ADD_TO_VISIT_TIME)
  visits['visit_time_epoch'] = pd.to_datetime(visit_time_in_ns, unit='s', utc=True).map(lambda x: x.tz_convert(TIMEZONE_STRING))
else:
  assert(False and "Extension of database is invalid")

#### Process the domains

In [ ]:
visits['domain'] = visits['url'].apply(process_domain_normal)

visits

In [ ]:
visits.tail(n=10)

#### Load the Ratings

In [ ]:
ratings = pd.read_csv(RATING_FILE, index_col='domain')
ratings.head()

#### Filter the Data based on Time + Visit Duration/Score

In [ ]:
greaterthanStartTime = visits['visit_time_epoch'] >= START_TIME
lessThanEndTime = visits['visit_time_epoch'] <= END_TIME
visits_this_semester = visits[greaterthanStartTime & lessThanEndTime]

In [ ]:
if 'visit_duration' in visits_this_semester.columns:
  min_visit_option = visits_this_semester[visits_this_semester['visit_duration'] >= MIN_CHROME_VISIT_DURATION]
  min_visit_option = min_visit_option[min_visit_option['visit_duration'] < MAX_CHROME_VISIT_DURATION]
elif 'score' in visits_this_semester.columns:
  min_visit_option = visits_this_semester[visits_this_semester['score'] >= MIN_SAFARI_SCORE]
else:
  min_visit_option = visits_this_semester

In [ ]:

final_data = min_visit_option.join(ratings[['manual_rating']], on="domain", how='left')
final_data['productivity_scale'] = final_data['manual_rating'].map(productivity_to_int_map)
final_data['time_of_day'] = final_data['visit_time_epoch'].dt.hour * 60 * 60 + final_data['visit_time_epoch'].dt.minute * 60 + final_data['visit_time_epoch'].dt.second
final_data.head()

#### Fill empty productivity scores with 0

In [ ]:
# Remove N/A
refined_data = final_data.copy()
refined_data['productivity_scale'] = refined_data['productivity_scale'].fillna(0)

### Graph 1: Productivity

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(refined_data['time_of_day'], refined_data['productivity_scale'], 'r.', alpha=0.05)
plt.xticks(np.arange(0, 60 * 60 * 24, step=60 * 60), np.arange(0, 24, step=1)) # Hours
plt.title("Productivity Level over Time \n {}".format(CUSTOM_PARAMS_FOR_GRAPH))
plt.xlabel("Time (Hour)")
plt.ylabel("Productivity level")
# plt.xticks(np.arange(0, 60 * 60 * 24, step=60 * 30), np.arange(0, 24, step=0.5))

if SAVE_GRAPH_1:
  plt.savefig("../plots/XX-filtered-data--a-week-of-data--{}.png".format(CUSTOM_PARAMS_FOR_FILE))

## Graph 2

### Work for Graph 2: Distracted Analysis

In [ ]:
only_distracted = refined_data[refined_data['productivity_scale'] < 0.5].copy()
only_distracted['half_hour'] = get_half_hour(only_distracted['time_of_day'])

only_distracted_agg = only_distracted.groupby('half_hour').agg('sum').reset_index()

### Graph 2

In [ ]:
plt.figure(figsize=(12, 8))
plt.plot(only_distracted_agg['half_hour'], only_distracted_agg['productivity_scale'], 'r-')
plt.plot(only_distracted_agg['half_hour'], only_distracted_agg['productivity_scale'], 'r.')
plt.xticks(np.arange(0, 48, 2), np.arange(0, 24, step=1)) # Half Hours
plt.title("Distracted Level throughout the day: \n ({})".format(CUSTOM_PARAMS_FOR_GRAPH))
plt.xlabel("Time (Hour)")
plt.ylabel("Sum of Productivity Level (-1 for Distracted. -2 for Very Distracted)")

if SAVE_GRAPH_2:
  plt.savefig("../plots/XX-filtered-data--a-week-of-data--{}.png".format(CUSTOM_PARAMS_FOR_FILE))

## Graph 3: Account for the visit duration

### Preprocessing (for Chrome only browsers)

In [ ]:
if DB_TYPE == TYPE_IS_CHROME:
  adj_prod_pre = refined_data.copy()


In [ ]:
if DB_TYPE == TYPE_IS_CHROME:
  # adj_prod = adj_prod_pre
  NUMBER_OF_INCREMENTS = 24 * (60 / 30) # 2 => half hour
  DAY_IN_SECONDS = 60 * 60 * 24
  ROUND_TO = DAY_IN_SECONDS / NUMBER_OF_INCREMENTS

  adj_prod_pre['adjusted_time'] = adj_prod_pre[['visit_duration', 'visit_time_epoch']].apply(
    lambda x: np.arange(0, x['visit_duration'] // 1e6, ROUND_TO) + x['visit_time_epoch'].value // 1e9, axis=1)

  adj_prod = adj_prod_pre.explode('adjusted_time')
  adj_prod.dropna(subset=['adjusted_time'], inplace=True)
else:
  adj_prod = refined_data.copy()


In [ ]:
if DB_TYPE == TYPE_IS_CHROME:
  adj_prod['adjusted_datetime'] = pd.to_datetime(adj_prod['adjusted_time'], unit='s', utc=True).map(lambda x: x.tz_convert(TIMEZONE_STRING))

  adj_prod['time_of_day'] = adj_prod['adjusted_datetime'].dt.hour * 60 * 60 \
    + adj_prod['adjusted_datetime'].dt.minute * 60 \
    + adj_prod['adjusted_datetime'].dt.second

In [ ]:
adj_prod['half_hour'] = get_half_hour(adj_prod['time_of_day'])

### Graph 3.1: Only Distracted Data

In [ ]:
adj_prod_distracted = adj_prod[adj_prod['productivity_scale'] < 0.5]

adj_prod_distracted_agg = adj_prod_distracted.groupby('half_hour').agg('sum').reset_index()


In [ ]:
if TYPE_IS_CHROME:
  plt.figure(figsize=(12, 8))
  plt.plot(adj_prod_distracted_agg['half_hour'], adj_prod_distracted_agg['productivity_scale'], 'r-')
  plt.plot(adj_prod_distracted_agg['half_hour'], adj_prod_distracted_agg['productivity_scale'], 'r.')
  plt.xticks(np.arange(0, NUMBER_OF_INCREMENTS, NUMBER_OF_INCREMENTS/24), np.arange(0, 24, step=1)) # Half Hours
  plt.title("Adjusted Productivity while Accouting for Visit Duration: \n ({})".format(CUSTOM_PARAMS_FOR_GRAPH))
  plt.xlabel("Time (Hour)")
  plt.ylabel("Sum of Productivity Level (-1 for Distracted. -2 for Very Distracted)")

  if SAVE_GRAPH_3:
    plt.savefig("../plots/XX-filtered-data--a-week-of-data--{}.png".format(CUSTOM_PARAMS_FOR_FILE))

### Graph 3.2: With Productive Data

In [ ]:
adj_prod['adjusted_prod'] = adj_prod['productivity_scale']
positive_sum = adj_prod[adj_prod['productivity_scale'] > 0]['productivity_scale'].sum()
adj_prod.loc[adj_prod['productivity_scale'] > 0, 'adjusted_prod'] /= positive_sum

negative_count = adj_prod[adj_prod['productivity_scale'] < 0]['productivity_scale'].sum()
adj_prod.loc[adj_prod['productivity_scale'] < 0, 'adjusted_prod'] /= -negative_count


In [ ]:
adj_prod_agg = adj_prod.groupby('half_hour').agg('sum').reset_index()


In [ ]:
plt.figure(figsize=(7, 5))
plt.plot(adj_prod_agg['half_hour'], adj_prod_agg['adjusted_prod'], 'r-')
plt.plot(adj_prod_agg['half_hour'], adj_prod_agg['adjusted_prod'], 'r.')
plt.axhline(y=0, color='b', linestyle='--', alpha=0.5)
plt.xticks(np.arange(0, NUMBER_OF_INCREMENTS, NUMBER_OF_INCREMENTS/24), np.arange(0, 24, step=1)) # Half Hours
plt.title("Adjusted Productivity while Accouting for Visit Duration: \n ({})".format(CUSTOM_PARAMS_FOR_GRAPH))
plt.xlabel("Time (Hour)")
plt.ylabel("Sum of Adjusted Productivity Level")

if SAVE_GRAPH_4:
  plt.savefig("../plots/XX-filtered-data--a-week-of-data--{}.png".format(CUSTOM_PARAMS_FOR_FILE))

## Graph 4: Final, using Seaborn confidence

In [ ]:
resetted_index = adj_prod.reset_index().drop('index', axis=1)

### Graph 4.1: View separately

In [ ]:
productive_data_agg = resetted_index[resetted_index['productivity_scale'] >= 0]
# \
#   .groupby('half_hour').agg('sum').reset_index()
  
unproductive_data_agg = resetted_index[resetted_index['productivity_scale'] <= 0]
# \
  # .groupby('half_hour').agg('sum').reset_index()


In [ ]:
plt.figure(figsize=(7, 5))
seaborn.lineplot(
    x='half_hour',
    y='adjusted_prod',
    data=productive_data_agg,
    color='green',
    marker='.',
)
seaborn.lineplot(
    x='half_hour',
    y='adjusted_prod',
    data=unproductive_data_agg,
    color='orange',
    marker='.',
)
# plt.plot(productive_data_agg['half_hour'], productive_data_agg['adjusted_prod'], 'g-')
# plt.plot(productive_data_agg['half_hour'], productive_data_agg['adjusted_prod'], 'g.')
# plt.plot(unproductive_data_agg['half_hour'], unproductive_data_agg['adjusted_prod'], 'y-')
# plt.plot(unproductive_data_agg['half_hour'], unproductive_data_agg['adjusted_prod'], 'y.')
plt.axhline(y=0, color='b', linestyle='--', alpha=0.5)
plt.xticks(np.arange(0, NUMBER_OF_INCREMENTS, NUMBER_OF_INCREMENTS/48), np.arange(0, 48, step=1)//2) # Half Hours

ax = plt.gca()
for x in ax.xaxis.get_ticklabels()[1::2]:
  x.set_visible(False)

plt.title("Normalized Productivity (NOT Accounting for Visit Duration): \n ({})".format(CUSTOM_PARAMS_FOR_GRAPH))
plt.xlabel("Time (Hour)")
plt.ylabel("Mean + 95 confidence interval of Normalized Productivity")

if SAVE_GRAPH_5:
  plt.savefig("../plots/XX-final-results--confidence-95-separate--{}--small".format(CUSTOM_PARAMS_FOR_FILE))

In [ ]:
adj_prod['adjusted_prod'] = adj_prod['productivity_scale']
positive_sum = adj_prod[adj_prod['productivity_scale'] > 0]['productivity_scale'].sum()
adj_prod.loc[adj_prod['productivity_scale'] > 0, 'adjusted_prod'] /= positive_sum

negative_count = adj_prod[adj_prod['productivity_scale'] < 0]['productivity_scale'].sum()
adj_prod.loc[adj_prod['productivity_scale'] < 0, 'adjusted_prod'] /= -negative_count


### 4.2 View Both Together

In [ ]:
START_INCREMENTS_AT = 12

In [ ]:
cut_off_data = resetted_index[resetted_index['half_hour'] >= START_INCREMENTS_AT]

In [ ]:
plt.figure(figsize=(7, 5))
seaborn.lineplot(
    x='half_hour',
    y='adjusted_prod',
    data=cut_off_data,
    color='red',
    marker='.',
)
plt.axhline(y=0, color='b', linestyle='--', alpha=0.5)
plt.xticks(np.arange(START_INCREMENTS_AT, NUMBER_OF_INCREMENTS, (NUMBER_OF_INCREMENTS - START_INCREMENTS_AT)/(48 - START_INCREMENTS_AT)), np.arange(START_INCREMENTS_AT, 48, step=1)//2) # Half Hours
# plt.xticks(np.arange(0, NUMBER_OF_INCREMENTS, NUMBER_OF_INCREMENTS/48), np.arange(0, 48, step=1)//2) # Half Hours

ax = plt.gca()
for x in ax.xaxis.get_ticklabels()[1::2]:
  x.set_visible(False)

plt.title("Normalized Productivity (NOT Accounting for Visit Duration): \n ({})".format(CUSTOM_PARAMS_FOR_GRAPH))
plt.xlabel("Time (Hour)")
plt.ylabel("Mean + 95 confidence interval of Normalized Productivity")

if SAVE_GRAPH_6:
    plt.savefig("../plots/XX-filtered-data--a-week-of-data--{}.png".format(CUSTOM_PARAMS_FOR_FILE))